  #                     Applying Machine Learning on Checkers
##                       (using Least-Mean-Square rule for Weight-update)

In [1]:
import numpy as np
import random
import copy

    Features used to evaulate a move are - 
    1. Number of white pawns
    2. Number of white kings
    3. Number of black pawns
    4. Number of black kings
    5. Number of white pieces threatened
    6. Number of black pieces threatened


    Setting up the Design for the representing Checkers (8*8)

    1. Board represents the 8*8 grid where the game is played  
    2. 'b'  = Black Pawn , 'B' = Black King , 'w'  = White pawn , 'W' = White king
    3. Declare the required variables globally for ease of access
    4. Deltas represent the directions in which a piece of a given color can move

In [2]:
n_rows = 8
n_columns = 8
n_cells = n_rows * n_columns
n_pieces = 0
n_white_pawns = 0
n_white_kings = 0
n_black_pawns = 0
n_black_kings = 0
n_white_threatened = 0
n_black_threatened = 0
winner  = ''
board = np.chararray((n_rows+1 , n_columns+1))
weight = np.random.rand(7)

deltaX = {  'w': [1,1],
            'W': [-1,-1,1,1],
            'b': [-1,-1],
            'B': [-1,-1,1,1]
         }
deltaY = {  'w': [-1,1],
            'W': [-1,1,1,-1],
            'b': [-1,1],
            'B': [-1,1,1,-1],
         }
# It is a list of target-values(i.e. Board value after move i) for all moves made by us 
targetValue = []
moveList = []
X = {}    # X stores the StateVector for each bestMove made by us

wcount = 0
bcount = 0

    InitializeBoard sets up the Board in the starting state.

In [3]:

def initializeBoard():
    global   n_pieces , n_white_kings , n_white_pawns , n_black_kings , n_black_pawns , n_rows , n_columns
    board[:] = ''
    for i in range(1,4):
        for j in range(1,n_columns+1):
            if((i+j)&1):
                board[i][j] = 'w'
            else :
                board[i][j] = ''

    for i in range(6,n_rows+1):
        for j in range(1,n_columns+1):
            if((i+j)&1):
                board[i][j] = 'b'
            else :
                board[i][j] = ''
    n_pieces = 24
    n_white_pawns = 12
    n_white_kings = 0
    n_black_pawns = 12
    n_black_kings = 0
    return 

def printBoard(board):
    print()
    for i in range(1,9):
        for j in range(1,9):
            if(board[i][j] == ''):
                print(".", end = " ")
            else:
                print(board[i][j].decode('utf-8')  , end=" ")
        print()
    print()
initializeBoard()
printBoard(board)


. w . w . w . w 
w . w . w . w . 
. w . w . w . w 
. . . . . . . . 
. . . . . . . . 
b . b . b . b . 
. b . b . b . b 
b . b . b . b . 



    These are helper functions whose functions are very much obvious from their names. 
    Many helper functions are defined in this project for Better Readability of the code.

In [4]:


def isEmptyCell(x,y , board): # unoccupied cell
    return (board[x][y] == '')

def containsEnemyPiece(x,y,colorToMove, board):
    if(isEmptyCell(x,y, board)):
        return False
    pieceColor = board[x][y].decode('utf-8')
    return (pieceColor.lower() != colorToMove.lower())

def validCell(x,y): # A valid cell lies on the board 
    global n_rows , n_columns
    return (x>=1 and x<=n_rows and y>=1 and y<=n_columns )

In [5]:
def canBeCaptured(x1 ,y1 , x2 , y2 ,board): # Can we move from (x1,y1 ) to capture on (x2,y2) ?    
    color_from = board[x1][y1].decode('utf-8')
    if(board[x2][y2] == ''):
        return False
    if(board[x2][y2] != ''):
        color_between = board[x2][y2].decode('utf-8')
        if(color_from.lower() == color_between.lower()): # Same colored pieces cant be captured
            return False
        dx = x2-x1
        dy = y2-y1
        x3 = x2 + dx
        y3 = y2 + dy
        if(validCell(x3,y3) == False or isEmptyCell(x3,y3,board) == False): # if after capturing, you land on a cell which is invalid then return False
            return False
    
    return True # if none of the above satisfy then it can be captured



    getMoves():  is used to get a list of all possible moves a player can make .

In [6]:
#Returns a dictionary of moves where moves[i] contains all the valid destinations from each coordinate i . 
#Also i of of the form (x_from , y_from)
def getMoves(colorToMove):
    moves = {}
    for x in range(1,9):
        for y in range(1,9):
            if((x+y)&1 and board[x][y] != ''):
                pieceColor = board[x][y].decode('utf-8')
                if(pieceColor.lower() == colorToMove.lower()):
                    dx = deltaX[pieceColor]
                    dy = deltaY[pieceColor]
                    n_validMoves = len(dx)
                    curr_moves = []
                    for k in range(0,n_validMoves):
                        x1 = x + dx[k]
                        y1 = y + dy[k]
                        if(validCell(x1,y1) == True ):
                            if(isEmptyCell(x1,y1,board) == True):
                                curr_moves.append((x1,y1))
                            elif(canBeCaptured(x,y,x1,y1,board) == True): # Can (x1,y1) be captured ?
                                x2 = x1 + (x1-x)
                                y2 = y1 + (y1-y)
                                curr_moves.append((x2,y2))
                    
                    if(len(curr_moves) != 0 ):
                        moves[(x,y)] = curr_moves
                    
    return moves 



    currentGameState(): constantly checks the board to see if it resulted in a victoty or a loss or the game   is still going on

In [7]:
def noLegalMoves(colorToMove): # returns True if there are no legal moves for the colorToMove
    
    moves = getMoves(colorToMove)
    
    if(len(moves) == 0 ):
        return 1
    return 0;
    
def currentGameState(colorToMove): # returns the winner of the game if game is not Ongoing
    global n_black_kings , n_black_pawns , n_white_kings , n_white_pawns
   
    if(colorToMove == 'b'):
        if(n_black_pawns + n_black_kings == 0 or noLegalMoves('b') == True):
            return "White"
    
    else :
        if(n_white_pawns + n_white_kings == 0 or noLegalMoves('w') == True):
            return "Black"
    return "Ongoing"

    getRandomMove(): returns a random move from the set of moves

In [8]:
def switchTurn(currentPlayer):
    if(currentPlayer.lower() == 'b'):
        return 'w'
    return 'b'



def getTotalNumberOfMoves(moves):
    res = 0
    for i in moves:
        res += len(moves[i])
    return res

def getRandomMove(colorToMove):
    
    randomMove = []
    moves = getMoves( colorToMove)
    totalMoves = getTotalNumberOfMoves(moves)
    if(totalMoves == 0): #
        return randomMove
    #print("Total moves under consideration " , totalMoves)
    move_to_pick = random.randint(0,totalMoves-1) #picking a random move 
    #print("picked index is ",move_to_pick)
    
    count = 0
    for i in moves:
        for j in moves[i]:
            if(count == move_to_pick):
                randomMove.append(i)
                randomMove.append(j)
            count += 1
    #print("random move given is " , randomMove)
    return randomMove
    

getThreatenedCount(): returns the count of black and white pieces that can be captured for a given Board                         Configuration 

In [9]:
def isACaptureMove(move):
    return (abs(move[0][0] - move[1][0]) == 2)

def isACaptureMove(x_from , y_from  , x_to , y_to):
    return (abs(x_from - x_to) == 2)

def getThreatenedCount(board): #returns count of black and white pieces threatened for a given board configuration
    res = {'b' : 0 , 'w' : 0}
    colors = ['b' , 'w']
    for color in colors:
        for x in range(1,9):
            for y in range(1,9):
                if((x+y)&1 and board[x][y] != '' and (board[x][y].decode('utf-8')).lower() == color):
                    pieceColor = board[x][y].decode('utf-8') # could be pawn or a king of same color as color
                    dx = deltaX[pieceColor]
                    dy = deltaY[pieceColor]
                    n_moves = len(dx)
                    for i in range(0,n_moves):
                        x1 = x + dx[i]
                        y1 = y + dy[i]
                        if(validCell(x1,y1) and containsEnemyPiece(x1,y1,color, board) and canBeCaptured(x,y,x1,y1,board)):
                            if(color == 'b'):
                                res['w'] += 1
                            else:
                                res['b'] += 1

    return res


    getStateVectorAfterMove(): This is the single most important function in the process. It assigns a value                              to  each move based on the Board configuration that move leads to. It returns the                               State Vector which contains the values of all the 6 Features for a given move.

In [10]:
def getStateVectorAfterMove(move ):
    stateVector = [1]
    
    #print("get state vector of move ",move)
    x_from = move[0][0]
    y_from = move[0][1]
    x_to = move[1][0]
    y_to = move[1][1]
    
    color_from = 'b'
    
    
    global n_white_pawns , n_white_kings , n_black_kings , n_black_pawns , n_black_threatened , n_white_threatened
    
    n_curr_white_pawns = n_white_pawns
    n_curr_white_kings = n_white_kings
    n_curr_black_pawns = n_black_pawns
    n_curr_black_kings = n_black_kings
    n_curr_black_threatened = n_black_threatened
    n_curr_white_threatened = n_white_threatened
    
    new_board = copy.deepcopy(board)
    
    if(isACaptureMove(x_from , y_from  , x_to , y_to) == True): 
        #captured piece was in between these two coordinates
        #print("Analysing a capture move")
        x_between = int(x_from + (x_to-x_from)/2)
        y_between = int(y_from + (y_to - y_from)/2)
        if(board[x_between][y_between] != ''):
            
            color_between = board[x_between][y_between].decode('utf-8')
            if(color_between == color_between.lower()): #pawn captured
                if(color_between.lower() == 'w'):
                    n_curr_white_pawns-=1
                else:
                    n_curr_black_pawns-=1
            else: #king captured
                if(color_between.lower() == 'w'):
                    n_curr_white_kings-=1
                else:
                    n_curr_black_kings-=1
        
        #now we play the move on board to get new board
        new_board[x_between][y_between] = ''
        new_board[x_from][y_from] = ''
        new_board[x_to][y_to] = color_from
        
    else :
        #now we actually move the piece to get a new board config
        new_board[x_from][y_from] = ''
        new_board[x_to][y_from] = color_from  #this may create some issue (str to unciode issue maybe)
        
    if(color_from == 'b'):
        if(x_to == 1): #Black pawn promotes to a King
            n_curr_black_pawns -= 1
            n_curr_black_kings += 1
            new_board[x_to][y_to] = 'B'
                
    elif(color_from == 'w'):#white pawn promotes to a king
        if(x_to == 8):
            new_board[x_to][y_to] = 'W'
            n_curr_white_pawns -= 1
            n_curr_white_kings += 1    
        
    threatenedMap = getThreatenedCount(new_board)
    n_curr_black_threatened = threatenedMap['b']
    n_curr_white_threatened = threatenedMap['w']
            
    stateVector.append(n_curr_white_pawns)
    stateVector.append(n_curr_white_kings)
    stateVector.append(n_curr_black_pawns)
    stateVector.append(n_curr_black_kings)
    stateVector.append(n_curr_black_threatened)
    stateVector.append(n_curr_white_threatened)
    #print(len(stateVector))
    return stateVector


    getTargetValue(move) :  This calculates the value of each move by using value = dot(weight, stateVector)

    getMapping(moves): This a mapping of each move to its TargetValue (i.e. the value of dot product of weight                             and StateVector)
    processMoves(moves): This just returns moves in a different easy to use format

In [11]:
def getTargetValue(move):
    X = getStateVectorAfterMove(move)
    res = np.dot(weight , X)
    return (res)

def getMapping(moves): #takes in the moves array which has been processed
    mapping = {}
    for i in range(0, len(moves)):
        targetValue = getTargetValue(moves[i])
        mapping[i] = targetValue
    return mapping

def processMoves(moves): # returns a list of moves where each moves[i] = [(x_from , y_from) , (x_to , y_to)]
    #Basically converts moves into a simple array of moves
    moves_prime = []
    index = 0
    for i in moves:
        for j in moves[i]:
            currMove = []
            currMove.append(i)
            currMove.append(j)
            moves_prime.append(currMove)
    return moves_prime


    getBestMove(color) : returns the move with the highest targetValue on the basis of our Machine Learning                          algo

In [12]:
def getBestMove(colorToMove): #returns [(x1,y1) , [x2,y2]] where piece on x1,y1 goes to x2,y2
    
    if(colorToMove == 'w'): # computer plays as White and picks a random move from the set of possible moves
        res = getRandomMove(colorToMove)
        moveList.append(res)
        return res
    
    # We make the black player's move
    
    moves = getMoves(colorToMove)
    
    moves_prime = processMoves(moves)
    
    move_to_target_value_mapping = getMapping(moves_prime)
    
    bestMoveIndex = max(move_to_target_value_mapping , key = move_to_target_value_mapping.get)
    targetValueOfBestMove = move_to_target_value_mapping[bestMoveIndex]
    
    bestMove = moves_prime[bestMoveIndex]
    
    # Both X and targetValue are required for weight update
    X[len(targetValue)] = getStateVectorAfterMove(bestMove)  
    targetValue.append(targetValueOfBestMove)
    moveList.append(bestMove)
    
    return bestMove
    
    
    

    playGivenMove(move): So after we get the best move for a player, we need to play it on the board . This       function helps us to do so. It also updates the global variables, thus keeping board up-to-date.            

In [13]:
def playGivenMove(move):
    x_from = move[0][0]
    y_from = move[0][1]
    x_to = move[1][0]
    y_to = move[1][1]
    
    color_from = board[x_from][y_from].decode('utf-8')
    global n_white_pawns , n_white_kings , n_black_kings , n_black_pawns , n_black_threatened , n_white_threatened
    
    
    if(isACaptureMove(x_from , y_from  , x_to , y_to)): 
        #captured piece was in between the above two coordinates
        x_between = int(x_from + (x_to-x_from)/2)
        y_between = int(y_from + (y_to - y_from)/2)
        color_between = board[x_between][y_between].decode('utf-8')
        if(color_between == color_between.lower()): #pawn captured
            if(color_between.lower() == 'w'):
                n_white_pawns-=1
            else:
                n_black_pawns-=1
        else: #king captured
            if(color_between.lower() == 'w'):
                n_white_kings-=1
            else:
                n_black_kings-=1
        
        #now we play the move on board to get new board
        board[x_between][y_between] = ''
        board[x_from][y_from] = ''
        board[x_to][y_to] = color_from
        
        
    else :
        #now we actually move the piece to get a new board config
        board[x_from][y_from] = ''
        board[x_to][y_to] = color_from  #this may create some issue (str to unciode issue maybe)
    
    if(color_from == 'b'):
        if(x_to == 1): #Black pawn promotes to a King
            n_black_pawns -= 1
            n_black_kings += 1
            board[x_to][y_to] = 'B'
                
    elif(color_from == 'w'):#white pawn promotes to a king
        if(x_to == 8):
            board[x_to][y_to] = 'W'
            n_white_pawns -= 1
            n_white_kings += 1
    
    threatenedMap = getThreatenedCount(board)
    n_black_threatened = threatenedMap['b']
    n_white_threatened = threatenedMap['w']
    
    return 
    
    

    updateWeights(): This is the back-propagation part. Here we update the weight using LMS rule. 
    
    delta_weight = (targetValue[next-best-Move] - targetValue[current-Move]) * (leanringRate)  *                                     (winningFactor) * (stateVector[cur-move])
    Winning factor : This is used because we dont want to change weights that much if we end up winning .
               
        

In [14]:
def updateWeights():
    global weight
    n_moves = len(targetValue)  #number of moves made by black during the game
    winner = currentGameState(board)
    winner_factor = 0
    if(winner == 'b'):
        winner_factor = 0.5
    else:
        winner_factor = 1
    
    for i in range(0,n_moves-1):
        delta = targetValue[i+1] - targetValue[i]
        X = getStateVectorAfterMove(moveList[i])
        for j in range(1,7):
            weight[j] += learningRate * delta * X[j] * winner_factor
    
    return
    

    simulateGame(): This is used to play a complete game

In [15]:


def simulateGame():
    targetValue.clear()
    moveList.clear()
    board[:] = ''
    initializeBoard()
    
    global wcount ,bcount
    currentPlayer = 'b'
    while(currentGameState(currentPlayer) == "Ongoing"):
        bestMove = getBestMove(currentPlayer)
        playGivenMove(bestMove)
        currentPlayer = switchTurn(currentPlayer)
    
    
    global winner
    if(currentPlayer == 'b'):
        winner = "White"
    else:
        winner = "Black"
        
    if(winner == 'White'):
        wcount += 1
    else :
        bcount += 1
        
    print(winner , " won in " , len(targetValue) , " moves")
   
    

    Train() : To train the model for given epochs and learning rate

In [16]:
def train(learningRate , epochs):
    print("MODEL TRAINING BEGINS :")
    print()
    print("Weights before training : " ,weight)
    print()
    global wcount , bcount
    wcount = 0
    bcount = 0
    for i in range(0,epochs):
        simulateGame()
        updateWeights()
    
    print()
    print("MODEL TRAINING ENDS ")

In [31]:
learningRate = 0.05
epochs = 10
train(learningRate , epochs)
print("Weights after training : " ,weight)

MODEL TRAINING BEGINS :

Weights before training :  [   0.19319867  -14.36404423   41.76201469   32.97992508 -131.99276637
    3.09798955    1.38345998]

Black  won in  117  moves
White  won in  207  moves
Black  won in  22  moves
White  won in  211  moves
Black  won in  260  moves
Black  won in  186  moves
Black  won in  88  moves
Black  won in  169  moves
Black  won in  94  moves
Black  won in  231  moves

MODEL TRAINING ENDS 
Weights after training :  [ 1.93198673e-01 -5.39747797e+00 -5.64180234e+01  7.39104826e+00
 -4.78103964e+02  3.14191705e+00  1.49327207e+00]


In [26]:
def test(weight , n_total_games):
    count = 0;
    for i in range(n_total_games):
        print("Game " , i+1," :" ,end = " ")
        simulateGame()
        if(winner == 'Black'):
            count += 1
    print("Games won by us (Black) : " , count)
    print("Total games played ", n_total_games)
    print("Accuracy ", (count/n_total_games)*100 , " %")

    Now we Test By playing as Black against the Computer(which choses random moves) . We play 10 games , 
    and lets see how much we win!

In [32]:
test(weight , 20)

Game  1  : Black  won in  77  moves
Game  2  : Black  won in  110  moves
Game  3  : Black  won in  29  moves
Game  4  : Black  won in  47  moves
Game  5  : White  won in  81  moves
Game  6  : Black  won in  71  moves
Game  7  : Black  won in  49  moves
Game  8  : White  won in  223  moves
Game  9  : Black  won in  33  moves
Game  10  : Black  won in  132  moves
Game  11  : Black  won in  26  moves
Game  12  : Black  won in  94  moves
Game  13  : Black  won in  57  moves
Game  14  : Black  won in  196  moves
Game  15  : White  won in  220  moves
Game  16  : Black  won in  116  moves
Game  17  : Black  won in  240  moves
Game  18  : Black  won in  30  moves
Game  19  : Black  won in  74  moves
Game  20  : White  won in  98  moves
Games won by us (Black) :  16
Total games played  20
Accuracy  80.0  %


    So, finally after testing, we found out that we won 80% of our games.